In [1]:
%pip install ../../

Processing /home/hyang/SpuCo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
ERROR: Package 'spuco' requires a different Python: 3.8.5 not in '>=3.10.0'
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [4]:
from spuco.utils import set_seed

set_seed(0)

ModuleNotFoundError: No module named 'spuco'

In [3]:
from wilds import get_dataset
from spuco.utils import initialize_bert_transform

# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="civilcomments", download=True, root_dir="/data")
transform = initialize_bert_transform(model="distilbert-base-uncased", max_token_length=300)

# Get the training set
train_data = dataset.get_subset(
    "train",
    transform=transform,
)

# Get the training set
test_data = dataset.get_subset(
    "test",
    transform=transform,
)

In [4]:
from spuco.utils import WILDSDatasetWrapper

trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="identity_any", verbose=True)
testset = WILDSDatasetWrapper(dataset=test_data, metadata_spurious_label="identity_any", verbose=True)

Partitioning data indices into groups: 100%|██████████| 133782/133782 [00:00<00:00, 3278503.19it/s]


In [5]:
trainset.group_weights

{(1, 1): 0.06904229142351638,
 (1, 0): 0.044380347757565844,
 (0, 0): 0.5338576706636238,
 (0, 1): 0.35271969015529403}

In [6]:
from spuco.models import model_factory 

model = model_factory("distilbert", (0,0,0), trainset.num_classes).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBert: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from torch.optim import AdamW
from spuco.invariant_train import ERM 

lr = 1e-5
weight_decay = 0.01
num_epochs = 1
batch_size = 16
max_grad_norm = 1.0

no_decay = ['bias', 'LayerNorm.weight']
params = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(
    params,
    lr=lr, 
    weight_decay=weight_decay)

from transformers import get_linear_schedule_with_warmup
n_train_steps = len(trainset) // batch_size * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_training_steps=n_train_steps,
    num_warmup_steps=0)

erm = ERM(
    model=model,
    num_epochs=num_epochs,
    trainset=trainset,
    batch_size=batch_size,
    optimizer=optimizer,
    lr_scheduler=scheduler,
    max_grad_norm=max_grad_norm,
    device=device,
    verbose=True,
)
erm.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()